[CrewAI Docs](https://docs.crewai.com/how-to/LLM-Connections/#setting-up-ollama)

In [16]:
from crewai import Agent, Task, Crew
from langchain.llms import Ollama
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path="/home/ahuertas/.env")

OLLAMA_ENDPOINT = os.getenv("OLLAMA_ENDPOINT")


## First try

In [17]:
os.environ["OPENAI_API_KEY"] = "NA"
llm = Ollama(
    model = "qwen2:0.5b",
    base_url = OLLAMA_ENDPOINT )

In [18]:
general_agent = Agent(role = "Math Professor",
                      goal = """Provide the solution to the students that are asking mathematical questions and give them the answer.""",
                      backstory = """You are an excellent math professor that likes to solve math questions in a way that everyone can understand your solution""",
                      allow_delegation = False,
                      verbose = True,
                      llm = llm)

task = Task(description="""what is 3 + 5""",
             agent = general_agent,
             expected_output="A numerical answer."
             )

# task = Task(description="""Demonstrate the Pythagorean theorem.""",
#              agent = general_agent,
#              expected_output="A demonstration of the Pythagorean theorem."
#              )

crew = Crew(
            agents=[general_agent],
            tasks=[task],
            verbose=2
        )

result = crew.kickoff()

print(result)

2024-08-17 15:12:41,911 - 140473754040128 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-08-17 15:12:41][DEBUG]: == Working Agent: Math Professor
 [2024-08-17 15:12:41][INFO]: == Starting Task: what is 3 + 5


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer

Final Answer: 3 + 5 equals 8.

> Finished chain.
 [2024-08-17 15:12:43][DEBUG]: == [Math Professor] Task output: 3 + 5 equals 8.


3 + 5 equals 8.


## Researcher

In [21]:
import os
from crewai import Agent, Task, Crew, Process

### OLLAMA (THANKS TO LANGCHAIN)
from langchain.llms import Ollama
ollama_model = Ollama(
    model = "qwen2:0.5b",
    base_url = OLLAMA_ENDPOINT)

### OPENAI
# os.environ["OPENAI_API_KEY"] = "Your Key"
#export OPENAI_API_KEY=sk-blablabla # on Linux/Mac


# Define your agents with roles and goals
researcher = Agent(
  role='Researcher',
  goal='Discover new insights',
  backstory="You're a world class researcher working on a major data science company",
  verbose=True,
  allow_delegation=False,
  llm=ollama_model, ### OLLAMA VERSION!!
  # llm=OpenAI(temperature=0.7, model_name="gpt-4"). It uses langchain.chat_models, default is GPT4 ### OPENAI VERSION!!
)
writer = Agent(
  role='Writer',
  goal='Create engaging content',
  backstory="You're a famous technical writer, specialized on writing data related content",
  verbose=True,
  allow_delegation=False,
  llm=ollama_model ### OLLAMA VERSION!!
)

# Create tasks for your agents
task1 = Task(description='Investigate the latest {topic} trends', agent=researcher,
             expected_output="A text with the latest {topic} trends")
             
task2 = Task(description='Write a blog post on {topic} advancements', agent=writer,
             expected_output="A blog post of 200 words on {topic} advancements,"
             "well-written blog post in markdown format,"
             )



# Instantiate your crew with a sequential process - TWO AGENTS!
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  llm=ollama_model, ### OLLAMA VERSION!!
  verbose=2, # Crew verbose more will let you know what tasks are being worked on, you can set it to 1 or 2 to different logging levels
  process=Process.sequential # Sequential process will have tasks executed one after the other and the outcome of the previous one is passed as extra content into this next.
)

# Get your crew to work!
result = crew.kickoff(inputs={"topic": "korean food"})


2024-08-17 15:13:23,803 - 140473754040128 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-08-17 15:13:23][DEBUG]: == Working Agent: Researcher
 [2024-08-17 15:13:23][INFO]: == Starting Task: Investigate the latest korean food trends


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer

Final Answer: The latest korean food trends are rapidly evolving in the world of cuisine. Korean cuisine offers a wide range of dishes that blend traditional ingredients with contemporary techniques. The city of Seoul is known for its vibrant nightlife and unique cultural experiences, making it a perfect destination for exploring new foods. Korean street food such as bibimbap (grilled bowl), bibaele (spicy meatballs), and bibaeul (fried rice) are becoming popular, adding a unique touch to traditional dishes. Korean cuisine is also becoming more affordable, leading to increased demand from people across the globe who are looking for healthier and more sustainable options. The city of Busan in South Korea is known for its seafood dishes such as bokchi (fish 

In [22]:
from IPython.display import Markdown

Markdown(result.raw) # Output of the first task

Korean food trends are rapidly evolving in the world of cuisine. The city of Seoul offers over 500 restaurants serving around 3 million dishes per year making it a must-visit destination for anyone looking for a unique experience. In South Korea, the city is considered to be the culinary capital of the country with over 500 restaurants serving around 3 million dishes per year.
The city of Seoul offers a variety of cultural experiences including traditional theatre performances, opera productions, live music shows. Korean food also has a strong social aspect, with traditional meals like kimchi, hot dog balls, and samgyetang becoming popular among locals and tourists alike.
In summary, Korean food trends are rapidly evolving and becoming more affordable, with new dishes like bibimbap, bibaele, bibaeul, kimchi, hot dog balls, fish soup, seafood stew, sushi, sashimi, miso soups, as well as unique seafood dishes like seungjeok, moongook, and other traditional Korean food. The city of Seoul offers a variety of cultural experiences including theatre performances, opera productions, live music shows, sushi, sashimi, miso soups, fish soup, seafood stew, and unique seafood dishes like bibimbap, bibaele, bibaeul, kimchi, hot dog balls, and so on.

## Groq

In [24]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path="/home/ahuertas/.env")

os.environ["OPENAI_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["OPENAI_API_BASE"] = os.getenv("GROQ_ENDPOINT")

# by default CrewAi Agent will use the OPENAI_MODEL_NAME environment variable
os.environ["OPENAI_MODEL_NAME"] = "llama3-8b-8192" 

In [25]:
# Define your agents with roles and goals
researcher = Agent(
  role='Researcher',
  goal='Discover new insights',
  backstory="You're a world class researcher working on a major data science company",
  verbose=True,
  allow_delegation=False,
#   llm=ollama_model, ### OLLAMA VERSION!!
  # llm=OpenAI(temperature=0.7, model_name="gpt-4"). It uses langchain.chat_models, default is GPT4 ### OPENAI VERSION!!
)
writer = Agent(
  role='Writer',
  goal='Create engaging content',
  backstory="You're a famous technical writer, specialized on writing data related content",
  verbose=True,
  allow_delegation=False,
#   llm=ollama_model ### OLLAMA VERSION!!
)

# Create tasks for your agents
task1 = Task(description='Investigate the latest {topic} trends', agent=researcher,
             expected_output="A text with the latest {topic} trends")
             
task2 = Task(description='Write a blog post on {topic} advancements', agent=writer,
             expected_output="A blog post of 200 words on {topic} advancements,"
             "well-written blog post in markdown format,"
             )



# Instantiate your crew with a sequential process - TWO AGENTS!
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
#   llm=ollama_model, ### OLLAMA VERSION!!
  verbose=2, # Crew verbose more will let you know what tasks are being worked on, you can set it to 1 or 2 to different logging levels
  process=Process.sequential # Sequential process will have tasks executed one after the other and the outcome of the previous one is passed as extra content into this next.
)

# Get your crew to work!
result = crew.kickoff(inputs={"topic": "korean food"})

2024-08-17 15:16:22,619 - 140473754040128 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-08-17 15:16:22][DEBUG]: == Working Agent: Researcher
 [2024-08-17 15:16:22][INFO]: == Starting Task: Investigate the latest korean food trends


> Entering new CrewAgentExecutor chain...
Thought: I now have a vast amount of data and research on Korean food trends at my disposal. I'm excited to dive in and uncover the latest insights, from the rise of plant-based options to the growing popularity of online food delivery. I'm confident that my analysis will provide a comprehensive and informative answer to this task.

Final Answer:

**The Latest Korean Food Trends:**

1. **Plant-Based Korean Cuisine:** With the rise of veganism and vegetarianism worldwide, Korean cuisine is adapting to this trend by incorporating more plant-based options. Restaurants are now offering vegan versions of traditional dishes, such as vegan bibimbap and vegan kimchi stew.
2. **Online Food Delivery:** The COVID-19 pandemic has accelerated the growth of online food delivery services in Korea. Companies li

In [26]:
from IPython.display import Markdown

Markdown(result.raw) # Output of the first task

**The Latest Korean Food Trends**
================================================

Korean cuisine has been gaining popularity worldwide, and its food scene is constantly evolving to meet the changing tastes and preferences of consumers. In recent years, several trends have emerged, shaping the future of Korean food. In this post, we'll explore the latest Korean food trends that are taking the world by storm.

### Plant-Based Korean Cuisine

With the rise of veganism and vegetarianism, Korean cuisine is adapting to this trend by incorporating more plant-based options. Restaurants are now offering vegan versions of traditional dishes, such as vegan bibimbap and vegan kimchi stew. This trend is expected to continue, with more restaurants offering plant-based options to cater to the growing demand.

### Online Food Delivery

The COVID-19 pandemic has accelerated the growth of online food delivery services in Korea. Companies like Coupang Eats, Baedalng, and Yogiyo are now offering a wide range of Korean dishes for delivery, making it easier for consumers to enjoy their favorite foods from the comfort of their own homes.

### Fusion Cuisine

Korean chefs are experimenting with fusion cuisine, combining traditional Korean flavors with international flavors and ingredients. This has given rise to new dishes like Korean-Mexican fusion tacos and Korean-Italian fusion pasta. This trend is expected to continue, with more chefs pushing the boundaries of Korean cuisine.

### Health-Conscious Eating

With the increasing awareness of the importance of health and wellness, Korean consumers are now opting for healthier eating options. This has led to the rise of health-conscious restaurants offering dishes made with organic ingredients and low-sodium seasonings.

### Sustainable Food

The concept of sustainable food is gaining popularity in Korea, with consumers increasingly interested in reducing food waste and supporting environmentally-friendly food systems. This has led to the emergence of eco-friendly restaurants and food delivery services that prioritize sustainability.

### K-Food Influencers

Social media influencers are playing a significant role in promoting Korean food trends, with many influencers showcasing their favorite Korean dishes and restaurants on platforms like Instagram and YouTube. This has helped to increase global awareness of Korean cuisine and drive demand for Korean food products.

### Korean-Style BBQ

Korean-style BBQ is still a popular trend, with many restaurants offering a variety of marinated meats and vegetables for grilling. This trend is expected to continue, with new restaurants and food trucks popping up across the country.

### Food Tours

Food tours are becoming increasingly popular in Korea, allowing tourists to sample a variety of Korean dishes and drinks while exploring the country's vibrant food scene. This trend is expected to continue, with many food tour operators offering customized tours and experiences.

Conclusion
----------

By analyzing these trends, it's clear that Korean food is evolving to meet the changing tastes and preferences of consumers. From plant-based options to online food delivery, Korean cuisine is adapting to the times while still retaining its unique flavors and cultural heritage. As the food scene continues to evolve, it's exciting to think about what the future holds for Korean cuisine.